In [4]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [10]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ],
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]


Error HH305: Unrecognized param --dataset_hash

For more info go to https://hardhat.org/HH305 or run Hardhat with --show-stack-traces


In [ ]:
zkml 

In [7]:
zkml add_bounty

Hashed inputs: 
14797455496207951391356508759149962584765968173479481191220882411966396840571n
Your Public Key: 
[
  20309760273953462500460369289927875375249161946486270897491509466557239169362n,
  4525867885453937990771346103622727204887718697147799398393210539764294025357n
]
Your Private Key: 
6355843196048741294034033711733592807513788875688648108428086728521504690935n
Current Balance
9509.978692845


In [8]:
zkml claim_bounty

Mean Squared Error actual:  0.11230295713531464
... quantized  18403
Mean Squared Error simulated:  0.11232375022889581895
... quantized  18406
Running Circuit ...
18406
Proof took 17.986 s
INFO: OK!
Paying 0x2546BcD3c84621e976D8185a91A922aE77ECEc30
With balance
10245.0
Your Public Key: 
[
  15125318766048368015918493405811853593843487344911443496030427796228303715156n,
  1337958943696657164700511549158913248996052066677825392636507599789718093268n
]
Your Private Key: 
19785431885146699345963608677700261499629304261029292666957242780310686520144n
Current Balance
10294.0


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
# Import Dataset from sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
from sklearn.datasets import load_iris # Load Iris Data

iris = load_iris() # Creating pd DataFrames

iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
target_df = pd.DataFrame(data= iris.target, columns= ['species'])

def converter(specie):
    if specie == 0:
        return 'setosa'
    elif specie == 1:
        return 'versicolor'
    else:
        return 'virginica'

target_df['species'] = target_df['species'].apply(converter)# Concatenate the DataFrames
iris_df = pd.concat([iris_df, target_df], axis= 1)

In [3]:
# Converting Objects to Numerical dtype
iris_df.drop('species', axis= 1, inplace= True)
target_df = pd.DataFrame(columns= ['species'], data= iris.target)
iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
y= iris_df['sepal length (cm)']

# Splitting the Dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.13, random_state= 101)

# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X_train, y_train)

# Making Predictions
lr.predict(X_test)
pred = lr.predict(X_test)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))

mse = mean_squared_error(y_test, pred)

Mean Squared Error: 0.1123029571353145


In [13]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
X = X_test.values[:]
Yt_expected = y_test.values[:].reshape(-1, 1)
mse_target = 0.11232375

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('model/X.npy',X)
np.save('model/Y.npy',Yt_expected)

In [15]:
Y_expected = np.matmul(X_test, lr.coef_) + lr.intercept_

assert (Y_expected == lr.predict(X_test)).all()

In [16]:
print(X.max(), X.min())
print(W.max(), W.min())
print(y.values[:].max(), y.values[:].min())
print(np.matmul(X_test, W).max(), np.matmul(X_test, W).min())
print((y_test - pred).max(), (y_test - pred).min())
print(((y_test - pred) ** 2).max(), ((y_test - pred) ** 2).min())
print(b)

6.7 0.0
0.7565732740231967 -0.49234541667216947
7.9 4.3
0    5.558082
dtype: float64 0    2.913389
dtype: float64
0.5143332457856635 -0.553112438773117
0.3059333699255451 0.000844615740033696
[[1.76203114]]


In [17]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = mse_target
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

In [21]:
!python3 scripts/quantize.py --mode model --settings settings.json --model model

GEMM Input Type (m,p) (p,n) (n,n) int0 int1 int2 int3 int4 int5 int6 int7
gemm assertion passed
error assertion passed
mse assertion passed
Mean Squared Error actual:  0.11230295713531464
quantized  18403
Mean Squared Error simulated:  0.11232375022889581895
quantized  18406


In [22]:
!python3 scripts/quantize.py --mode dataset --settings settings.json --model model

In [18]:
quant_model('model', 'settings.json')

GEMM Input Type (m,p) (p,n) (n,n) int0 int1 int2 int3 int4 int5 int6 int7
gemm assertion passed
error assertion passed
mse assertion passed
Mean Squared Error actual:  0.1123029571353145
quantized  18403
Mean Squared Error simulated:  0.11232375022889581895
quantized  18406


In [19]:
quant_dataset('model', 'settings.json')